In [1]:
%cd /ibex/user/slimhy/PADS/code
%reload_ext autoreload
%set_env CUBLAS_WORKSPACE_CONFIG=:4096:8
"""
Extracting features into HDF5 files for each split.
"""
import argparse
import torch

import util.misc as misc
import models.s2vs as ae_mods


def get_args_parser():
    parser = argparse.ArgumentParser("Extracting Features", add_help=False)

    # Model parameters
    parser.add_argument(
        "--batch_size",
        default=32,
        type=int,
        help="Batch size per GPU"
        " (effective batch size is batch_size * accum_iter * # gpus",
    )
    parser.add_argument(
        "--text_model_name",
        type=str,
        help="Text model name to use",
    )
    parser.add_argument(
        "--ae",
        type=str,
        metavar="MODEL",
        help="Name of autoencoder",
    )
    parser.add_argument(
        "--ae-latent-dim",
        type=int,
        default=512*8,
        help="AE latent dimension",
    )
    parser.add_argument(
        "--ae_pth",
        required=True,
        help="Autoencoder checkpoint"
    )
    parser.add_argument(
        "--point_cloud_size",
        default=2048,
        type=int,
        help="input size"
    )
    parser.add_argument(
        "--fetch_keys",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--use_embeds",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--intensity_loss",
        action="store_true",
        default=False,
        help="Contrastive edit intensity loss using ground-truth labels.",
    )

    # Dataset parameters
    parser.add_argument(
        "--dataset",
        type=str,
        choices=["graphedits"],
        help="dataset name",
    )
    parser.add_argument(
        "--data_path",
        type=str,
        help="dataset path",
    )
    parser.add_argument(
        "--data_type",
        type=str,
        help="dataset type",
    )
    parser.add_argument(
        "--max_edge_level",
        default=None,
        type=int,
        help="maximum edge level to use",
    )
    parser.add_argument(
        "--device", default="cuda", help="device to use for training / testing"
    )
    parser.add_argument("--seed", default=0, type=int)
    parser.add_argument("--num_workers", default=60, type=int)
    parser.add_argument(
        "--pin_mem",
        action="store_true",
        help="Pin CPU memory in DataLoader for more efficient "
        "(sometimes) transfer to GPU.",
    )

    return parser


# Set dummy arg string to debug the parser
call_string = """--ae_pth ckpt/ae_m512.pth \
    --ae kl_d512_m512_l8 \
    --ae-latent-dim 4096 \
    --data_path /ibex/project/c2273/PADS/3DCoMPaT \
    --batch_size 32 \
    --num_workers 8 \
    --device cuda"""
    

# Parse the arguments
args = get_args_parser()
args = args.parse_args(call_string.split())

# --------------------
device = torch.device(args.device)
misc.set_all_seeds(args.seed)
torch.backends.cudnn.benchmark = True
# --------------------

# Initialize and load autoencoder
ae = ae_mods.__dict__[args.ae]()
ae.load_state_dict(torch.load(args.ae_pth, map_location="cpu")["model"])
ae = torch.compile(ae.eval().to(device), mode="max-autotune")


/ibex/user/slimhy/PADS/code
env: CUBLAS_WORKSPACE_CONFIG=:4096:8
Set seed to 0


In [2]:
from datasets.latents import ShapeLatentDataset, ComposedPairedShapesLoader

class PairType():
    NO_ROT_PAIR = "rand_no_rot,rand_no_rot"
    PART_DROP = "part_drop,orig"

# Create your datasets
dataset_train = ShapeLatentDataset(args.data_path, split="train", shuffle_parts=True, filter_n_ids=2)
dataset_val = ShapeLatentDataset(args.data_path, split="test", shuffle_parts=False, filter_n_ids=2)

# Create the DataLoader using the sampler
data_loader_train = ComposedPairedShapesLoader(
    dataset_train,
    batch_size=4,
    pair_types_list=[PairType.NO_ROT_PAIR],
    num_workers=0,
    shuffle=True,
    use_distributed=False
) 

In [3]:
import torch
from torch import nn
from models.modules import (
    Attention,
    DiagonalGaussianDistribution,
    GEGLU,
    PreNorm,
    StackedAttentionBlocks
)
from models.partqueries import PartEmbed


class PartQueriesGenerator(nn.Module):
    """
    Generating a set of part-aware latents
    from a set of part bounding boxes and part labels: "part queries".
    """

    def __init__(
        self,
        dim=512,
        latent_dim=128,
        max_parts=24,
        heads=8,
        in_heads=1,
        dim_head=64,
        depth=2,
        weight_tie_layers=False,
        use_attention_masking=True,
    ):
        super().__init__()

        self.latent_dim = latent_dim
        self.max_parts = max_parts
        self.use_attention_masking = use_attention_masking

        # Part Embeddings
        self.part_embed = PartEmbed(dim)
        self.embed_proj = nn.Sequential(
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, dim),
        )

        # Input Cross-Attention Block
        self.in_encode = PreNorm(
            dim, Attention(dim, dim, heads=in_heads, dim_head=dim), context_dim=dim
        )
        
        # Replace repeat with learnable projection
        self.latent_proj = nn.Sequential(
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, dim),
        )

        # Stacked Attention Layers
        self.encoder_layers = nn.Identity()
        if depth > 0:
            self.encoder_layers = StackedAttentionBlocks(
                dim, depth, heads, dim_head, weight_tie_layers
            )

        # Compress latents to latent dimension
        self.compress_latents = nn.Sequential(
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, latent_dim),
        )

    def forward(self, latents, part_bbs, part_labels, batch_mask):
        """
        :param latents:     B x 512 x 8
        :param part_bbs:    B x 24 x 4 x 3
        :param part_labels: B x 24
        :param batch_mask:  B x 24
        """
        # Embed part labels and bounding boxes
        part_embeds, labels_embed, bb_embeds = self.part_embed(
            part_bbs, part_labels, batch_mask
        )
        
        # Apply learnable projection instead of repeat
        latents_kv = latents.transpose(1, 2).repeat(1, 3, 1) # B x 24 x 512
        latents_kv = self.latent_proj(latents_kv)
        
        # Concatenate part embeddings with latents
        part_embeds = self.embed_proj(part_embeds) # B x 24 x 512

        # Encode part embeddings
        mask = batch_mask if self.use_attention_masking else None
        x = self.in_encode(part_embeds, context=latents_kv, mask=mask) # B x 24 x 512
        x = self.encoder_layers(x)               # B x 24 x 512
        part_latents = self.compress_latents(x)  # B x 24 x 512
        return part_latents, part_embeds

    @property
    def device(self):
        return next(self.parameters()).device


class PartAwareAE(nn.Module):
    def __init__(
        self,
        dim=512,
        latent_dim=128,
        max_parts=24,
        heads=8,
        in_heads=1,
        dim_head=64,
        depth=2,
        weight_tie_layers=False,
        use_attention_masking=True,
    ):
        super().__init__()

        self.is_vae = False
        self.dim = dim
        self.latent_dim = latent_dim
        self.use_attention_masking = use_attention_masking

        # Encoder
        self.encoder = PartQueriesGenerator(
            dim=dim,
            latent_dim=latent_dim,
            max_parts=max_parts,
            heads=heads,
            in_heads=in_heads,
            dim_head=dim_head,
            depth=depth,
            weight_tie_layers=weight_tie_layers,
            use_attention_masking=use_attention_masking,
        )
        
        # Decoder components
        self.in_decode = PreNorm(
            dim, Attention(dim, dim, heads=in_heads, dim_head=dim), context_dim=dim
        )
        
        self.decode_proj = nn.Linear(dim * 2, dim)
        self.out_proj = nn.Linear(24, 8)

        # Stacked Attention Layers for decoder
        self.decoder_layers = StackedAttentionBlocks(
            dim, depth, heads, dim_head, weight_tie_layers
        )

        # Expand latents
        self.expand_latents = nn.Sequential(
            nn.Linear(latent_dim, dim),
            GEGLU(),
            nn.Linear(dim // 2, dim),
        )

    def decode(self, part_latents, bb_embeds, batch_mask):
        x = self.expand_latents(part_latents)
        mask = batch_mask if self.use_attention_masking else None
        x = torch.cat([x, bb_embeds], dim=-1)
        x = self.decode_proj(x)
        x = self.in_decode(x, mask=mask) #, context=bb_embeds
        x = self.decoder_layers(x)
        x = self.out_proj(x.transpose(1, 2))
        return x

    def forward(self, latents, part_bbs, part_labels, batch_mask):
        part_latents, bb_embeds = self.encoder(
            latents, part_bbs, part_labels, batch_mask
        )
        logits = self.decode(part_latents, bb_embeds, batch_mask)

        return logits, part_latents

    @property
    def device(self):
        return next(self.parameters()).device


class PartAwareVAE(PartAwareAE):
    def __init__(
        self,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)
        self.is_vae = True
        self.mean_fc = nn.Linear(self.latent_dim, self.latent_dim)
        self.logvar_fc = nn.Linear(self.latent_dim, self.latent_dim)

    def encode(self, latents, part_bbs, part_labels, batch_mask, deterministic=False):
        part_latents, bb_embeds = self.encoder(
            latents, part_bbs, part_labels, batch_mask
        )

        mean = self.mean_fc(part_latents)
        logvar = self.logvar_fc(part_latents)

        posterior = DiagonalGaussianDistribution(
            mean, logvar, deterministic=deterministic, no_reduction=True
        )
        part_latents = posterior.sample()
        kl = posterior.kl()

        return kl, part_latents, bb_embeds

    def forward(self, latents, part_bbs, part_labels, batch_mask, deterministic=False):
        kl, part_latents, bb_embeds = self.encode(
            latents, part_bbs, part_labels, batch_mask, deterministic=deterministic
        )
        logits = self.decode(part_latents, bb_embeds, batch_mask).squeeze(-1)

        return logits, kl, part_latents


In [4]:
import torch
from torch import nn

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)


def assert_close(tensor1, tensor2, rtol=1e-5, atol=1e-5):
    assert torch.allclose(tensor1, tensor2, rtol=rtol, atol=atol), \
        f"Tensors are not close: \n{tensor1}\n{tensor2}"


def test_latent_permutation_invariance(model):
    batch_size, num_parts = 1,  24
    
    latents = torch.rand(batch_size, 512, 8)
    part_bbs = torch.rand(batch_size, num_parts, 4, 3)
    part_labels = torch.randint(0, 10, (batch_size, num_parts), dtype=torch.long)
    batch_mask = torch.ones(batch_size, num_parts).bool()
    
    l_0, p_l_0 = model(latents, part_bbs, part_labels, batch_mask)
    
    perm = torch.randperm(8)
    
    l_1, p_l_1 = model(latents[:, :, perm], part_bbs, part_labels, batch_mask)
    
    try:
        assert_close(l_0, l_1)
        print("Reconstruction invariant to latents permutation: PASSED")
    except AssertionError as e:
        print("Reconstruction invariant to latents permutation: FAILED")
        print(str(e))


def test_part_latents_equivariance(model):
    batch_size, num_parts = 1, 24
    
    latents = torch.rand(batch_size, 512, 8)
    part_bbs = torch.rand(batch_size, num_parts, 4, 3)
    part_labels = torch.randint(0, 10, (batch_size, num_parts), dtype=torch.long)
    batch_mask = torch.ones(batch_size, num_parts).bool()
    
    l_0, p_l_0 = model(latents, part_bbs, part_labels, batch_mask)
    
    perm = torch.randperm(num_parts)
    permuted_part_bbs = part_bbs[:, perm, :, :]
    permuted_part_labels = part_labels[:, perm]
    
    l_1, p_l_1 = model(
        latents,
        permuted_part_bbs,
        permuted_part_labels,
        batch_mask
    )

    try:
        assert_close(p_l_0[:, perm, :], p_l_1)
        print("Part latents equivariant to parts permutation: PASSED")
    except AssertionError as e:
        print("Part latents equivariant to parts permutation: FAILED")
        print(str(e))
        
        
def print_return_shapes(model):
    batch_size, num_parts = 1, 24
    
    latents = torch.rand(batch_size, 512, 8)
    part_bbs = torch.rand(batch_size, num_parts, 4, 3)
    part_labels = torch.randint(0, 10, (batch_size, num_parts), dtype=torch.long)
    batch_mask = torch.ones(batch_size, num_parts).bool()
    
    part_latents, part_embeds = model(latents, part_bbs, part_labels, batch_mask)
    print("Part latents shape:", part_latents.shape)
    print("Part embeddings shape:", part_embeds.shape)


# Initialize your model
model = PartAwareAE(
    dim=512,
    latent_dim=128,
    max_parts=24,
    heads=8,
    in_heads=1,
    dim_head=64,
    depth=2,
    use_attention_masking=True,
)

# Run the tests
print_return_shapes(model)
print()
test_latent_permutation_invariance(model)
print()
test_part_latents_equivariance(model)

Part latents shape: torch.Size([1, 512, 8])
Part embeddings shape: torch.Size([1, 24, 128])

Reconstruction invariant to latents permutation: PASSED

Part latents equivariant to parts permutation: PASSED


In [5]:
import torch.nn as nn
from torch.nn import functional as F
from losses.partvae import KLRecLoss, RecLoss, ScaleInvariantLoss, PartDropLoss
from schedulefree import AdamWScheduleFree


def get_losses():
    """
    Instantiate the losses.
    """
    return (
        RecLoss(),
        KLRecLoss(),
        ScaleInvariantLoss(),
        PartDropLoss(),
    )


def forward_pass(
    pvae,
    data_tuple,
    rec_loss,
    kl_rec_loss,
    scale_inv_loss,
    part_drop_loss,
    pair_types,
):
    """
    Compute a single forward pass of the model.
    """
    # Unpack the data tuple
    pair_types, (l_a, bb_a, bb_l_a, meta_a), (l_b, bb_b, bb_l_b, meta_b) = data_tuple
    device = pvae.device

    # Compute the mask from batch labels
    mask_a = (bb_l_a != -1).to(device)  # B x 24
    mask_b = (bb_l_b != -1).to(device)  # B x 24

    l_a, l_b = l_a.to(device), l_b.to(device)  # B x 8 x 512
    # l_a = l_a.transpose(1, 2)  # B x 512 x 8
    # l_b = l_b.transpose(1, 2)  # B x 512 x 8
    bb_a, bb_b = bb_a.to(device), bb_b.to(device)  # B x 24 x 4 x 3
    bb_l_a, bb_l_b = bb_l_a.to(device), bb_l_b.to(device)  # B x 24

    # Optionally compute the KL Reg loss
    if pvae.is_vae:
        logits_a, kl_a, part_latents_a = pvae(
            latents=l_a, part_bbs=bb_a, part_labels=bb_l_a, batch_mask=mask_a
        )
        logits_b, kl_b, part_latents_b = pvae(
            latents=l_b, part_bbs=bb_b, part_labels=bb_l_b, batch_mask=mask_b
        )

        # KL Reg loss
        kl_reg = kl_rec_loss(kl_a, mask=mask_a) + kl_rec_loss(kl_b, mask=mask_b)
        kl_reg /= 2.0
    else:
        logits_a, part_latents_a = pvae(
            latents=l_a, part_bbs=bb_a, part_labels=bb_l_a, batch_mask=mask_a
        )
        logits_b, part_latents_b = pvae(
            latents=l_b, part_bbs=bb_b, part_labels=bb_l_b, batch_mask=mask_b
        )
        kl_reg = torch.tensor(0.0).to(device)

    # L2 loss
    rec_loss = F.mse_loss(logits_a, l_a) + F.mse_loss(logits_b, l_b)
    # rec_loss = rec_loss(logits_a, l_a, transpose=True) + rec_loss(logits_b, l_b, transpose=True)
    rec_loss /= 2.0

    # if pair_types == PairType.NO_ROT_PAIR:
    #     inv_loss = scale_inv_loss(part_latents_a, part_latents_b, mask_a)
    # elif pair_types == PairType.PART_DROP:
    #     inv_loss = part_drop_loss(
    #         part_latents_a, part_latents_b, bb_a, bb_b, mask_a, mask_b
    #     )
        
    # Add part latents magnitude loss
    inv_loss = F.mse_loss(part_latents_b, torch.zeros_like(part_latents_b))
    inv_loss += F.mse_loss(part_latents_a, torch.zeros_like(part_latents_a))

    return {
        "kl_reg": kl_reg,
        "rec_loss": rec_loss,
        "inv_loss": inv_loss,
        "last_sample": (logits_a, l_a, bb_a, bb_l_a, mask_a),
    }


# Initialize the model
pvae = PartAwareVAE(
    dim=512,
    latent_dim=512,
    heads=8,
    dim_head=64,
    depth=4,
).to(device)
pvae = pvae.to(device)
pvae.train(True)

# Initialize the optimizer
optimizer = AdamWScheduleFree(
    pvae.parameters(), lr=1e-4, weight_decay=1e-5
)
optimizer.zero_grad()

rec_loss, kl_loss, scale_inv_loss, part_drop_loss = get_losses()

metric_logger = misc.MetricLogger(delimiter="  ")

for epoch in metric_logger.log_every(
    list(range(5000)), 100
):
    data_seen = False
    for data_tuple in data_loader_train:
        data_seen = True
        # Computing loss
        loss = forward_pass(
            pvae=pvae,
            data_tuple=data_tuple,
            rec_loss=rec_loss,
            kl_rec_loss=kl_loss,
            scale_inv_loss=scale_inv_loss,
            part_drop_loss=part_drop_loss,
            pair_types=PairType.PART_DROP,
        )
        total_loss = (
            0.005 * loss["kl_reg"]
            + 1. * loss["rec_loss"]
            + 0.01 * loss["inv_loss"]
        )
        data_seen = True
        
        accum_iter = 1

        # Backward pass
        total_loss /= accum_iter
        total_loss.backward()
        
        # Clip the gradients
        nn.utils.clip_grad_norm_(pvae.parameters(), 5.)

        optimizer.step()
        optimizer.zero_grad()

        # Log the losses
        loss_update = {
            "train_loss": float(total_loss.item()),
            "kl_reg": float(loss["kl_reg"].item()),
            "rec_loss": float(loss["rec_loss"].item()),
            "inv_loss": float(loss["inv_loss"].item()),
        }
        metric_logger.update(**loss_update)
    assert data_seen, "No data seen in the training loop."

  [   0/5000]  eta: 0:34:32  train_loss: 1.0466 (1.0466)  kl_reg: 4.1607 (4.1607)  rec_loss: 1.0055 (1.0055)  inv_loss: 2.0317 (2.0317)  time: 0.4144  data: 0.0000  max mem: 1221
  [ 100/5000]  eta: 0:10:35  train_loss: 0.4993 (0.6539)  kl_reg: 0.1299 (0.2921)  rec_loss: 0.4793 (0.6326)  inv_loss: 1.9591 (1.9747)  time: 0.1425  data: 0.0000  max mem: 1221
  [ 200/5000]  eta: 0:10:47  train_loss: 0.3136 (0.5149)  kl_reg: 0.0866 (0.1929)  rec_loss: 0.2938 (0.4943)  inv_loss: 1.9285 (1.9589)  time: 0.1406  data: 0.0000  max mem: 1221
  [ 300/5000]  eta: 0:10:43  train_loss: 0.2042 (0.4256)  kl_reg: 0.0800 (0.1631)  rec_loss: 0.1898 (0.4062)  inv_loss: 1.3665 (1.8593)  time: 0.1414  data: 0.0000  max mem: 1221
  [ 400/5000]  eta: 0:10:34  train_loss: 0.1299 (0.3584)  kl_reg: 0.0343 (0.1353)  rec_loss: 0.1162 (0.3404)  inv_loss: 1.3608 (1.7339)  time: 0.1414  data: 0.0000  max mem: 1221
  [ 500/5000]  eta: 0:10:20  train_loss: 0.1017 (0.3090)  kl_reg: 0.0290 (0.1145)  rec_loss: 0.0878 (0.29

In [6]:
import numpy as np
import trimesh
import k3d
from util import s2vs


def get_args_parser():
    parser = argparse.ArgumentParser("Autoencoder Visualization", add_help=False)
    parser.add_argument("--ae_pth", required=True, help="Autoencoder checkpoint")
    parser.add_argument("--ae", type=str, default="kl_d512_m512_l8", help="Name of autoencoder")
    parser.add_argument("--ae-latent-dim", type=int, default=4096, help="AE latent dimension")
    parser.add_argument("--batch_size", default=32, type=int, help="Batch size")
    parser.add_argument("--num_workers", default=8, type=int, help="Number of workers for data loading")
    parser.add_argument("--device", default="cuda", help="Device to use for computation")
    parser.add_argument("--seed", default=0, type=int, help="Random seed")
    parser.add_argument("--pin_mem", action="store_true", help="Pin CPU memory in DataLoader")
    return parser

def obb_to_corners(box_array):
    center, right, up, forward = [np.array(box_array[i]) for i in range(4)]
    corners = np.array([
        [-1, -1, -1], [ 1, -1, -1], [ 1,  1, -1], [-1,  1, -1],
        [-1, -1,  1], [ 1, -1,  1], [ 1,  1,  1], [-1,  1,  1]
    ])
    transform = np.column_stack((right, up, forward))
    return np.dot(corners, transform.T) + center

def create_trimesh_boxes(bounding_boxes):
    return [
        trimesh.util.concatenate([
            trimesh.primitives.Sphere(radius=0.01).apply_translation(corner)
            for corner in obb_to_corners(box)
        ])
        for box in bounding_boxes if not np.all(box == 0)
    ]

def load_checkpoint(model, checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    return model

def visualize_decoded_mesh(decoded_mesh, bounding_boxes):
    trimesh_boxes = create_trimesh_boxes(bounding_boxes)
    col_map = k3d.helpers.map_colors(np.arange(len(trimesh_boxes)), k3d.colormaps.basic_color_maps.Rainbow)
    
    plot = k3d.plot()
    plot += k3d.mesh(decoded_mesh.vertices, decoded_mesh.faces, color=0x0000ff, opacity=0.5)
    for k, bb_mesh in enumerate(trimesh_boxes):
        plot += k3d.mesh(bb_mesh.vertices, bb_mesh.faces, color=int(col_map[k]), opacity=0.5)
    return plot

def show_mesh(ae, logits_a, bb_a):
    assert logits_a.shape[0] == 1, "Can only visualize a single shape at a time"
    mesh = s2vs.decode_latents(ae, logits_a.to(ae.device), grid_density=128, batch_size=64**3)
    bounding_boxes = np.array(bb_a[0].cpu())
    plot = visualize_decoded_mesh(mesh.trimesh_mesh, bounding_boxes)
    return plot

In [7]:
with torch.inference_mode():
    logits_a, l_a, bb_a, bb_l_a, mask_a = loss["last_sample"]
    logits_a, _ = pvae(
        latents=l_a, part_bbs=bb_a, part_labels=bb_l_a, batch_mask=mask_a
    )

ValueError: too many values to unpack (expected 2)

In [ ]:
show_mesh(ae, logits_a[0].unsqueeze(0), bb_a)